In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
df_meta = pd.read_csv("TCGAbiolinks_BRCA.csv", header=0, index_col=0, skiprows=0)
df_meta.dropna(how="all", axis=1)
df_meta["birth_year"] = [(dt.datetime.now()+dt.timedelta(days=d)) if d!="" else np.nan for d in df_meta["days_to_birth"].fillna("")]
df_meta["death_year"] = [(dt.datetime.now()-dt.timedelta(days=d)) if d!="" else np.nan for d in df_meta["days_to_death"].fillna("")]
df_meta["age"] = (df_meta["death_year"] - df_meta["birth_year"]).apply(lambda timestamp: timestamp/dt.timedelta(days=365))
bins = np.linspace(df_meta["age"].min(),df_meta["age"].max(), 3)
df_meta["age_binned"] = pd.cut(df_meta["age"], bins=bins, labels = bins[:-1], include_lowest=True)
df_meta.columns

In [ ]:
df_tcgabiolinks = pd.read_csv("../TCGA_biolinks.csv", index_col=0).dropna(how="all", axis=1)
df_tcgabiolinks = df_tcgabiolinks[df_tcgabiolinks["cancer.type"]=="BRCA"].set_index("pan.samplesID")
df_tcgabiolinks["Subtype_Selected_Lum"]=df_tcgabiolinks["Subtype_Selected"]
df_tcgabiolinks["Subtype_Selected_Lum"].replace("BRCA.LumA","BRCA.Lum", inplace=True)
df_tcgabiolinks["Subtype_Selected_Lum"].replace("BRCA.LumB","BRCA.Lum", inplace=True)

In [ ]:
df_files=pd.read_csv("files_tcga.dat", index_col=2)
df_files = df_files.reset_index().set_index("cases.0.submitter_id")
df_files = df_files.transpose().append(df_meta.reindex(index=df_files.index).transpose()).transpose()
df_files = df_files.reset_index().set_index("cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id")
df_files = df_files.transpose().append(df_tcgabiolinks.reindex(index=df_files.index).transpose()).transpose()
df_files.fillna("unknown").set_index("file_name").to_csv("files.dat", index=True, header=True)

In [ ]:
df_files.groupby(["BRCA_Subtype_PAM50","Subtype_Selected_Lum", "Subtype_Selected"]).count()

In [ ]:
df=pd.read_csv("mainTable_all.csv", index_col=0, header=0).applymap(lambda x: np.log2(x+1))
df.info()

In [ ]:
dey = pd.read_csv("http://stephenslab.github.io/count-clustering/project/utilities/gene_names_all_gtex.txt", header=None)

In [ ]:
df=df.reindex(index=dey.values.ravel()[np.random.randint(0,len(dey), size=1000)])

In [ ]:
full_files = [c for c in df_files['file_name'].values if c in df.columns]

In [ ]:
df[full_files].dropna().astype(int).to_csv("mainTable.csv")

In [ ]:
df_files[df_files['file_name'].isin(full_files)].set_index('file_name').to_csv("files.dat")